In [4]:
import math
import random
import numpy as np
import time
import matplotlib
# matplotlib.use('TkAgg')
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from matplotlib.lines import Line2D

class Chromosome:
    def __init__(self,fitness_func,range=(-10,10),h=0.02,random_num=None):
        self.fitness_func = fitness_func
        self.length = math.ceil(math.log2(abs(range[1] - range[0]) / h + 1))
        self.h = h
        if (random_num is None):
            self.DNA = self.encode(random.uniform(*range))
        else:
            self.DNA = self.encode(random_num)

    def __repr__(self):
        return self.DNA
    
    def getFitness(self):
        x = self.decode()
        return self.fitness_func(x)

    def getLength(self):
        return self.length
    
    def encode(self, x):
        k = int(round((x + 10) / self.h)) ######
        return format(k, f'0{self.length}b')

    def decode(self):
        k = int(self.DNA,2)
        return k * self.h - 10 ######
            
    def __str__(self):
        return self.DNA

    def copy(self):
         new_chrom = Chromosome(...) 
         new_chrom.DNA = self.DNA
         return new_chrom
         
class Population:
    def __init__(self, size, h,r,fitness_func, population=[]):
        self.size = size
        self.fitness_func = fitness_func
        if not population:
            self.population = [Chromosome(range=r,h=h,fitness_func=self.fitness_func) for _ in range(self.size)]
        else:
            self.population = population

    def __repr__(self):
        return str(self.population)
        
    def __str__(self):
        return str(self.population)
        
    def __getitem__(self,index):
        return self.population[index]
        
    def __len__(self):
        return self.size

    def get_best_fit(self):
        return max(self.population,key=lambda chromo: chromo.getFitness())
        
class GeneticAlgorithm:
    def __init__(self,fitness_func,step=0.02, range=(-10,10),func_label=None):
        self.population_size = 1
        self.step = step
        self.range = range
        self.fitness_func = fitness_func
        self.T = 200 #start temperature
        self.initial_T = self.T 
        self.alpha = 0.95 #cooling rate
        self.t = 0.01
        self.best = {}
        self.best['fitness'] = -float('inf')
        self.best['chromo'] = None
        self.population = Population(size=self.population_size, h=self.step, r=self.range, fitness_func=self.fitness_func)
        self.counter = 0
        self.xhist = []
        self.yhist = []
        self.fig, self.ax = plt.subplots(figsize=(10, 6))
        self.func_label = func_label or self._get_function_label(fitness_func)
        x = np.linspace(self.range[0], self.range[1], 400)
        y = [self.fitness_func(xi) for xi in x]
        self.ax.plot(x, y, 'b-', label=self.func_label)
        self.current_line = Line2D([], [], color='r', linestyle='--', label='current x')
        self.ax.add_line(self.current_line)
        self.best_point, = self.ax.plot([], [], 'go', markersize=10, label='best x overtime')
        self.ax.set_xlim(self.range[0],self.range[1])
        self.ax.grid()
        self.ax.legend()

    def anim_init(self):
        self.current_line.set_xdata([np.nan]) 
        self.best_point.set_data([], [])
        return self.current_line, self.best_point

       
    def anneal_step(self,current):
        min_val = self.range[0]
        max_val = self.range[1]
        val = current.decode()
        exploration_strength = self.T / self.initial_T
        radius = (max_val - min_val) * (0.05 + 0.5 * exploration_strength)
        lower = max(val - radius, min_val)
        upper = min(val + radius, max_val)
        # new_val = Chromosome(random_num=random.uniform(lower, upper),fitness_func=self.fitness_func) #uniform distribution selection
        new_val = Chromosome(
        random_num=np.clip(random.gauss(val, radius), *self.range), #gaussian distribution selection
        fitness_func=self.fitness_func
    )
        # print(f'current {val} with fit {current.getFitness()}')
        # print(f'new {new_val.decode()} with fit {new_val.getFitness()}')
        deltaF = new_val.getFitness() - current.getFitness()
        # print(f'deltaF {deltaF}')
        if(deltaF > 0):
            # print("Accepted: better solution")
            population = Population(size=self.population_size,h=self.step,r=self.range,population=[new_val],fitness_func=self.fitness_func)
        else:
            normalized_delta = deltaF / (abs(current.getFitness()) + 1e-6)
            P = math.exp(normalized_delta / self.T)
            # print(f"P(accept worse) = {P:.4f}")
            if(random.random() < P):
                # print("Accepted: worse solution")
                population = Population(size=self.population_size,h=self.step,r=self.range,population=[new_val],fitness_func=self.fitness_func)
            else:
                # print("Rejected: keep current solution")
                population = self.population
        return population

    
    def __get_results(self):
        print(f"-----------------------\nmax found in x = {self.best['chromo'].decode()}")
        print(f"f(x) = {self.best['fitness']:.10f}")
        print(f'execution time: {self.end_time - self.start_time:.2f}с')
        

    def run(self):
        self.counter = 0
        self.start_time = time.time()
        max_frames = self.T - self.t + 1
        self._algorithm_generator()
        self._create_animation()
        
    def _algorithm_generator(self):
        print(f"\nPopulation size: {self.population_size} ")
        print(f'funcion: {self.func_label}')
        self.start_time = time.time()
        while self.T > self.t:
            best_chromo = self.population.get_best_fit()
            best_fit = best_chromo.getFitness()
            current_x = best_chromo.decode()
            current_y = best_chromo.getFitness()
            self.xhist.append(current_x)
            self.yhist.append(current_y)
            if best_fit > self.best['fitness']:
                self.best['fitness'] = best_fit
                self.best['chromo'] = best_chromo
            self.population = self.anneal_step(best_chromo)
            self.counter += 1
            self.T *= self.alpha
        
        self.end_time = time.time()
        self.__get_results()

    def _create_animation(self):
        def update(frame):
            current_x = self.xhist[frame]
            current_y = self.yhist[frame]
            self.current_line.set_data([current_x, current_x], [self.ax.get_ylim()[0], current_y])
            if self.best['chromo']:
                best_x = self.best['chromo'].decode()
                best_y = self.best['fitness']
                self.best_point.set_data([best_x], [best_y])
            self.ax.set_title(f'step: {frame}/{len(self.xhist)-1}\n'
                            f'current value: {self.yhist[frame]:.4f}\n'
                            f'best overtime value: {self.best["fitness"]:.4f}')
            return self.current_line, self.best_point
        self.ani = FuncAnimation(
            self.fig, update,
            frames=len(self.xhist),
            init_func=self.anim_init,
            interval=75, ### manipulates the speed
            blit=True,
            repeat=False
        )
    def _get_function_label(self, func):
        """Trying to get functions's source code"""
        try:
            import inspect
            source = inspect.getsource(func)
            # retrieving expression after return keyword
            if 'return' in source:
                expr = source.split('return')[-1].strip()
                return f"f(x) = {expr}"
        except:
            pass
        return "Target function"
        
functions = [
    # lambda x: (1.85 - x) * np.cos(3.5 * x - 0.5),
    # lambda x: -x**2 + 10 * np.sin(x) + 10,
    lambda x: np.sin(5 * x) * np.exp(-0.1 * x),
    # lambda x: -np.abs(x - 2) + np.sin(10 * x),
    # lambda x: np.cos(x) + np.cos(3 * x) / 2 + np.cos(5 * x) / 3
]
intervals = [
    # (-10, 10),
    # (-10, 10),
    (0, 30),
    # (-10, 20),
    # (-31.4, 31.4)
]

labels = [
    # "(1.85 - x) * cos(3.5x - 0.5)",
    # "-x² + 10sin(x) + 10",
    "sin(5x) * exp(-0.1x)"
    # "-|x - 2| + sin(10x)",
    # "cos(x) + cos(3x)/2 + cos(5x)/3"
]
for i, func in enumerate(functions):
    gen_alg = GeneticAlgorithm(step=0.02, range=intervals[i],fitness_func=func,func_label=labels[i])
    gen_alg.run()
    # display(HTML(gen_alg.ani.to_jshtml()))
    # plt.close(gen_alg.fig)  


Population size: 1 
funcion: sin(5x) * exp(-0.1x)
-----------------------
max found in x = 4.1
f(x) = 0.6615463423
execution time: 0.01с
